In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
!pip install -qq "transformers==4.38.0" --upgrade
!pip -qq install bitsandbytes accelerate datasets
!pip -qq install "torch>=2.1.1" # https://huggingface.co/google/gemma-7b/discussions/17

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
dataset = load_dataset('medmcqa', split='train').shuffle(seed=42)
dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [4]:
for question in dataset['question'][0:2]:
    chat = [
      { "role": "user", "content": question },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=32)
    display(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


'<bos><start_of_turn>user\nOPV can be used if vaccine l monitor is showing?<end_of_turn>\n<start_of_turn>model\nSure, here is the answer to the question:\n\nOPV (oral poliovirus vaccine) can be used if the vaccine monitor is showing. However, it'

'<bos><start_of_turn>user\nAverage gain of height in first year ?<end_of_turn>\n<start_of_turn>model\nThe average gain of height in the first year is around 2-3 inches.<eos>'